Dataset Focus: We would like to be able to cluster these accidents based on pertinent variables. Since we do not have data for both our labels of interest (i.e. accident or not), we will be performing unsupervised learning to try to group similar conditions together.

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns

In [4]:
data = pd.read_csv('../data/Traffic_Collision_Data_from_2010_to_Present.csv')

In [5]:
data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)"
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)"
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)"
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)"
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)"


In [6]:
data.shape  # 18 columns and over 500k observations

(570372, 18)

In [7]:
# Convert column names into something that is easier to work with (add underscores)\
for col in data.columns:
    tmp = ('_').join(col.lower().split())
    data.rename(columns={col: tmp}, inplace=True)

In [8]:
data.columns

Index(['dr_number', 'date_reported', 'date_occurred', 'time_occurred',
       'area_id', 'area_name', 'reporting_district', 'crime_code',
       'crime_code_description', 'mo_codes', 'victim_age', 'victim_sex',
       'victim_descent', 'premise_code', 'premise_description', 'address',
       'cross_street', 'location'],
      dtype='object')

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570372 entries, 0 to 570371
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   dr_number               570372 non-null  int64  
 1   date_reported           570372 non-null  object 
 2   date_occurred           570372 non-null  object 
 3   time_occurred           570372 non-null  int64  
 4   area_id                 570372 non-null  int64  
 5   area_name               570372 non-null  object 
 6   reporting_district      570372 non-null  int64  
 7   crime_code              570372 non-null  int64  
 8   crime_code_description  570372 non-null  object 
 9   mo_codes                483387 non-null  object 
 10  victim_age              487109 non-null  float64
 11  victim_sex              561234 non-null  object 
 12  victim_descent          560341 non-null  object 
 13  premise_code            569410 non-null  float64
 14  premise_description 

In [90]:
date_cols = ['date_reported', 'date_occurred'] #'time_occurred'] 

for col in date_cols:
    data[col] = pd.to_datetime(data[col])

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570372 entries, 0 to 570371
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   dr_number               570372 non-null  int64         
 1   date_reported           570372 non-null  datetime64[ns]
 2   date_occurred           570372 non-null  datetime64[ns]
 3   time_occurred           570372 non-null  int64         
 4   area_id                 570372 non-null  int64         
 5   area_name               570372 non-null  object        
 6   reporting_district      570372 non-null  int64         
 7   crime_code              570372 non-null  int64         
 8   crime_code_description  570372 non-null  object        
 9   mo_codes                483387 non-null  object        
 10  victim_age              487109 non-null  float64       
 11  victim_sex              561234 non-null  object        
 12  victim_descent          560341

In [95]:
data.isnull().sum()  # Number of NaN values for each variable

dr_number                     0
date_reported                 0
date_occurred                 0
time_occurred                 0
area_id                       0
area_name                     0
reporting_district            0
crime_code                    0
crime_code_description        0
mo_codes                  86985
victim_age                83263
victim_sex                 9138
victim_descent            10031
premise_code                962
premise_description         963
address                       0
cross_street              27224
location                      0
dtype: int64

In [92]:
data['time_occurred'].min(), data['time_occurred'].max()  # follows 24 hr clock

(1, 2359)

## EDA

In [93]:
data.describe()

,dr_number,time_occurred,area_id,reporting_district,crime_code,victim_age,premise_code
count,5.703720e+05,570372.000000,570372.000000,570372.000000,570372.0,487109.000000,569410.000000
mean,1.550780e+08,1356.473943,11.081726,1154.145354,997.0,41.230755,102.432546
std,3.216413e+07,598.677049,5.875726,588.751853,0.0,16.390919,23.451665
min,1.001000e+08,1.000000,1.000000,100.000000,997.0,10.000000,101.000000
25%,1.304141e+08,930.000000,6.000000,666.000000,997.0,28.000000,101.000000
50%,1.603106e+08,1430.000000,11.000000,1162.000000,997.0,38.000000,101.000000
75%,1.813112e+08,1820.000000,16.000000,1653.000000,997.0,51.000000,101.000000
max,2.221050e+08,2359.000000,21.000000,2199.000000,997.0,99.000000,970.000000


Based on these results we can drop crime_code as they all have the same value, and will not be distinguishing/predictive.

In [96]:
data.drop(columns='crime_code', inplace=True)

In [98]:
data.corr()

,dr_number,time_occurred,area_id,reporting_district,victim_age,premise_code
dr_number,1.000000,0.006642,0.022624,0.022209,-0.024146,-0.001113
time_occurred,0.006642,1.000000,0.010771,0.010899,-0.023836,-0.002825
area_id,0.022624,0.010771,1.000000,0.998925,0.009347,0.003122
reporting_district,0.022209,0.010899,0.998925,1.000000,0.009560,0.003242
victim_age,-0.024146,-0.023836,0.009347,0.009560,1.000000,0.009254
premise_code,-0.001113,-0.002825,0.003122,0.003242,0.009254,1.000000


Based on the correlation matrix, the only correlation is between `reporting_district` and `area_id`, as expected.

Therefore, we know for missing numeric variables (`victim_age`, and `premise_code`) that there are no significant correlations between other variables to use for predictive imputation. Therefore, the best course of action for dealing with these missing variables would be to remove them or use mean/median/mode imputation). Additionally, it would be wise to view all of these variables, and remove those that we do not deem as important/significant to the grouping.

For example, it would be smart to remove

In [52]:
nan_columns = data.isna().any()
cols_w_nan = data.columns[nan_columns].tolist()
cols_w_nan

['mo_codes',
 'victim_age',
 'victim_sex',
 'victim_descent',
 'premise_code',
 'premise_description',
 'cross_street']

570372 total observations and the above columns have NaN values that need to be adjusted/removed.